<a href="https://colab.research.google.com/github/AnjaliS2021/WildfireDetection/blob/main/wildfire_detection_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title ###### Licensed to the Apache Software Foundation (ASF), Version 2.0 (the "License")

# Licensed to the Apache Software Foundation (ASF) under one
# or more contributor license agreements. See the NOTICE file
# distributed with this work for additional information
# regarding copyright ownership. The ASF licenses this file
# to you under the Apache License, Version 2.0 (the
# "License"); you may not use this file except in compliance
# with the License. You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing,
# software distributed under the License is distributed on an
# "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
# KIND, either express or implied. See the License for the
# specific language governing permissions and limitations
# under the License.

#Introduction
Owner: Anjali Singh

Data resources
- wildfire_detection_dataset.ipynb

Modeling resources
- Earth Engine https://developers.google.com/earth-engine/guides/machine-learning
- U-Net https://keras.io/examples/vision/oxford_pets_image_segmentation/
- Segformer https://keras.io/examples/vision/segformer/

#🛰 Packages

In [ ]:
#@title Installs
!pip install --quiet transformers

In [ ]:
#@title Imports
# Keep the imports in sorted order.
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from tensorflow import keras

import gzip
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import sys

print(f'py version: {sys.version}')
print(f'tf version: {tf.__version__}')

In [ ]:
#@title Google Drive
from google.colab import drive
drive.mount('/content/drive')
wildfire_drive_path = '/content/drive/MyDrive/wildfire_detection'
print(os.listdir(wildfire_drive_path))

# 🧠 U-Net model

In [ ]:
#@title 📖 Read Dataset
class UnetDatasetBuilder:

  def __init__(self):
    self.image_size = None  # Will be updated from dataset.
    self.num_bands = 3
    self.min_fire_mask = 7
    self.max_fire_mask = 9
    self.fire_masks = self.get_fire_masks()
    self.num_classes = self.get_num_classes()
    self.max_frp = 2400
    self.batch_size = 16
    self.build_paths()
    self.build_dataset()

  def __str__(self):
    return '\n'.join([
      f'fire_masks: {self.fire_masks}',
      f'image size: {self.image_size}',
      f'batch size: {self.batch_size}',
      f'train dataset: {self.train_dataset.element_spec}',
      f'test dataset: {self.test_dataset.element_spec}',
      f'keras.backend.image_data_format: {tf.keras.backend.image_data_format()}',
    ])

  def build_paths(self):
    self.base_directory = wildfire_drive_path
    self.dataset_directory = 'dataset'
    self.dataset_path = os.path.join(
      self.base_directory, self.dataset_directory)
    self.train_path = os.path.join(self.dataset_path, 'train')
    self.test_path = os.path.join(self.dataset_path, 'test')

  def get_fire_masks(self):
    fire_masks = [0]
    fire_masks.extend(range(self.min_fire_mask, self.max_fire_mask+1))
    return fire_masks

  def get_num_classes(self):
    num_classes = max(self.fire_masks) + 1
    return num_classes

  def build_dataset(self):
    self.train_dataset = self.load_dataset(self.train_path)
    self.test_dataset = self.load_dataset(self.test_path)

  def load_dataset(self, dataset_path):
    # Use tf.data.TFRecordDataset to read the TFRecord files. It gets bytes for
    # each element. The read_example converts that into image and label tensors.
    filenames = tf.data.Dataset.list_files(
        f'{dataset_path}/*/part-*.tfrecord.gz')
    print(f'Loading num files: {len(list(filenames.as_numpy_iterator()))}')
    dataset = tf.data.TFRecordDataset(filenames, compression_type='GZIP')
    dataset = dataset.map(self.read_example, num_parallel_calls=tf.data.AUTOTUNE)
    self.set_image_size(dataset)
    dataset = dataset.map(self.update_example, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(self.batch_size).shuffle(
        buffer_size=16, reshuffle_each_iteration=True).prefetch(
            buffer_size=16)
    return dataset

  def read_example(self, serialized):
    features_dict = {
      'inputs': tf.io.FixedLenFeature([], tf.string),
      'labels': tf.io.FixedLenFeature([], tf.string),
    }
    example_tf = tf.io.parse_single_example(serialized, features_dict)
    inputs = tf.io.parse_tensor(example_tf['inputs'], tf.int32)
    labels = tf.io.parse_tensor(example_tf['labels'], tf.int64)
    # TensorFlow can't infer the shapes, so we set them explicitly.
    inputs.set_shape([None, None, self.num_bands])
    labels.set_shape([None, None, 1])
    example = {'inputs': inputs, 'labels': labels}
    return example

  def update_example(self, example):
    inputs = example['inputs']
    labels = example['labels']
    inputs, labels = self.normalize(inputs, labels)
    # Resize as expected by the U-Net.
    image_size = self.image_size
    inputs = tf.image.resize(inputs, (image_size, image_size),
                             method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    labels = tf.image.resize(labels, (image_size, image_size),
                             method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    example.update(self.format(inputs, labels))
    inputs = example['inputs']
    labels = example['labels']
    return inputs, labels

  def normalize(self, input_image, input_mask):
    input_image = tf.clip_by_value(input_image, 0, self.max_frp)
    input_image = tf.cast(input_image, tf.float32)
    scale = 255.0 / self.max_frp
    input_image *= scale
    input_mask = self.normalize_label(input_mask)
    return input_image, input_mask

  def normalize_label(self, input_mask):
    input_mask = tf.cast(input_mask, tf.int32)
    return input_mask

  def format(self, input_image, input_mask):
    return {'inputs': input_image, 'labels': input_mask}

  def set_image_size(self, dataset):
    self.image_size = 160 # This size is required by Unet.

  def debug_dataset(self):
    print('train dataset')
    dataset = self.train_dataset
    dataset = dataset.unbatch()
    for inputs, labels in dataset.take(1):
      print(f'inputs : {inputs.dtype.name} {inputs.shape}')
      print(inputs)
      print(f'labels : {labels.dtype.name} {labels.shape}')
      print(labels)

unet_dataset_builder = UnetDatasetBuilder()
print(unet_dataset_builder)
unet_dataset_builder.debug_dataset()

## 🏛 Model Architecture

In [ ]:
#@title Create U-Net
class UnetModelBuilder:

  def __init__(self):
    self.dataset_builder = unet_dataset_builder
    self.image_size = self.dataset_builder.image_size
    self.num_bands = self.dataset_builder.num_bands
    self.num_classes = self.dataset_builder.num_classes
    self.build_paths()
    self.build_model()
    self.compile_model()
    self.build_checkpoint()

  def __str__(self):
    return '\n'.join([
      f'num_bands: {self.num_bands}',
      f'num_classes: {self.num_classes}'
    ])

  def build_paths(self):
    self.base_directory = wildfire_drive_path
    self.model_directory = 'unet_model'
    self.model_path = os.path.join(self.base_directory, self.model_directory)
    self.checkpoint_directory = 'unet_checkpoint'
    self.checkpoint_path = os.path.join(
        self.base_directory, self.checkpoint_directory, 'unet')

  def build_model(self):
    inputs = keras.Input(shape=(self.image_size, self.image_size, self.num_bands))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = keras.layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = keras.layers.Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.UpSampling2D(2)(x)

        # Project residual
        residual = keras.layers.UpSampling2D(2)(previous_block_activation)
        residual = keras.layers.Conv2D(filters, 1, padding="same")(residual)
        x = keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = keras.layers.Conv2D(self.num_classes, 3, activation="softmax", padding="same")(
        x
    )

    # Define the model
    self.model = keras.Model(inputs, outputs, name="UNet")
    self.model.summary()

  def compile_model(self):
    unet8s_optimizer = keras.optimizers.Adam()
    unet8s_loss = keras.losses.SparseCategoricalCrossentropy()
    # Maintain mIOU and Pixel-wise Accuracy as metrics
    self.model.compile(
        optimizer=unet8s_optimizer,
        loss=unet8s_loss,
        metrics=[
            keras.metrics.MeanIoU(num_classes=self.num_classes, sparse_y_pred=False),
            keras.metrics.SparseCategoricalAccuracy(),
        ],
    )

  def build_checkpoint(self):
    self.checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        self.checkpoint_path, save_weights_only=True)
    checkpoint_directory = os.path.dirname(self.checkpoint_path)
    latest_checkpoint_path = tf.train.latest_checkpoint(checkpoint_directory)
    if latest_checkpoint_path:
      print(f'Load checkpoint: {latest_checkpoint_path}')
      self.model.load_weights(latest_checkpoint_path)

  def train_and_save_model(self, epochs=1):
    self.build_checkpoint()
    self.train_model(epochs)
    self.save_model()

  def train_model(self, epochs=1):
    train_dataset = self.dataset_builder.train_dataset
    test_dataset = self.dataset_builder.test_dataset
    self.history = self.model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=epochs,
        callbacks=[self.checkpoint_callback]
    )

  def save_model(self):
    self.model.save(self.model_path)

  def load_model(self):
    self.model = tf.keras.models.load_model(self.model_path)

  def run_predict(self, inputs, labels=None):
    if labels is not None:
      labels = labels[0]
      if tf.is_tensor(labels):
        labels = labels.numpy().astype(np.uint8)
    probabilities = self.model.predict(inputs)
    probabilities = probabilities[0]
    if tf.is_tensor(probabilities):
      probabilities = probabilities.numpy()
    predictions = probabilities.argmax(axis=-1).astype(np.uint8)
    if labels is not None:
      predictions.resize(labels.shape)
    return predictions, labels

unet_model_builder = UnetModelBuilder()
print(unet_model_builder)

In [ ]:
#@title Train the model.
#unet_model_builder.train_and_save_model(epochs=1)
print(os.listdir(unet_model_builder.model_path))

## 🔮 Model Predictions

In [ ]:
#@title Analyze Model
class UnetModelAnalyzer:

  def __init__(self, dataset_builder=unet_dataset_builder,
               model_builder=unet_model_builder):
    self.dataset_builder = dataset_builder
    self.model_builder = model_builder

  def run_analysis(self):
    all_examples = self.build_dataset()
    all_predictions = []
    all_labels = []
    for inputs, labels in all_examples:
      predictions, labels = self.run_predict(inputs, labels)
      all_predictions.append(predictions)
      all_labels.append(labels)
    self.show_matrix(all_predictions, all_labels)

  def build_dataset(self, max_examples=5):
    all_examples = []
    dataset = self.dataset_builder.test_dataset.unbatch().batch(1)
    for inputs, labels in dataset:
      if max_examples <= 0:
        break
      max_examples -= 1
      all_examples.append([inputs, labels])
    return all_examples

  def run_predict(self, inputs, labels):
    predictions, labels = self.model_builder.run_predict(inputs, labels)
    predictions = predictions.flatten()
    labels = labels.flatten()
    return predictions, labels

  def show_matrix(self, all_predictions, all_labels):
    y_true = np.concatenate(all_labels, axis=None)
    y_pred = np.concatenate(all_predictions, axis=None)
    fire_masks = self.dataset_builder.fire_masks

    cfr = classification_report(y_true, y_pred, labels=fire_masks, zero_division=0)
    print(cfr)
    cfm = confusion_matrix(y_true, y_pred, labels=fire_masks)
    disp = ConfusionMatrixDisplay(confusion_matrix=cfm,
                                  display_labels=fire_masks)
    cfm_norm = confusion_matrix(y_true, y_pred, labels=fire_masks, normalize='true')
    disp_norm = ConfusionMatrixDisplay(confusion_matrix=cfm_norm, display_labels=fire_masks)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    disp.plot(ax=ax1)
    disp_norm.plot(ax=ax2)
    plt.show()

unet_model_analyzer = UnetModelAnalyzer()
unet_model_analyzer.run_analysis()


# 🤗 Vision Transformer
Details at SegFormer and Hugging Face Transformers
* https://keras.io/examples/vision/segformer/

In [ ]:
#@title Imports
from transformers import TFSegformerForSemanticSegmentation

In [ ]:
#@title 📖 Read Dataset
class SegFormerDatasetBuilder(UnetDatasetBuilder):

  def __init__(self):
    super(SegFormerDatasetBuilder, self).__init__()
    self.build_paths()
    self.build_dataset()

  def get_num_classes(self):
    num_classes = 0
    for mask in self.fire_masks:
      if mask > 0:
        num_classes += 1
    return num_classes

  def set_image_size(self, dataset):
    self.image_size = 512  # This size is required by SegFormer.

  def update_example(self, example):
    inputs = example['inputs']
    labels = example['labels']
    # Resize as expected by the SegFormer.
    image_size = self.image_size
    inputs = tf.image.resize(inputs, (image_size, image_size),
                             method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    labels = tf.image.resize(labels, (image_size, image_size),
                             method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # SegFormer expects normalized pixel values.
    inputs, labels = self.normalize(inputs, labels)
    example = self.format(inputs, labels)
    return example

  def normalize(self, input_image, input_mask):
    input_image = tf.clip_by_value(input_image, 0, self.max_frp)
    input_image = tf.cast(input_image, tf.float32)
    input_image /= self.max_frp  # range: 0 to 1.
    # These values are specified by the SegFormer.
    pretrained_mean = tf.constant([0.485, 0.456, 0.406])
    pretrained_std = tf.constant([0.229, 0.224, 0.225])
    input_image = (input_image - pretrained_mean) / tf.maximum(
        pretrained_std, tf.keras.backend.epsilon())
    input_mask = tf.cast(input_mask, tf.int32)
    return input_image, input_mask

  def format(self, input_image, input_mask):
    # Transpose the images such that they are in 'channels_first' format. This
    # is to make them compatible with the SegFormer model from Hugging Face
    # Transformers.
    input_image = tf.transpose(input_image, (2, 0, 1))
    return {'pixel_values': input_image, 'labels': tf.squeeze(input_mask)}

  def debug_dataset(self):
    print('train dataset')
    dataset = self.train_dataset
    dataset = dataset.unbatch()
    for example in dataset.take(1):
      print(example)

segformer_dataset_builder = SegFormerDatasetBuilder()
print(segformer_dataset_builder)
segformer_dataset_builder.debug_dataset()

## 🏛 Model architecture

In [ ]:
#@title Create SegFormer
class SegFormerModelBuilder:

  def __init__(self):
    self.dataset_builder = segformer_dataset_builder
    self.fire_masks = self.dataset_builder.fire_masks
    self.image_size = self.dataset_builder.image_size
    self.checkpoint = "nvidia/mit-b0"
    self.build_paths()
    self.build_model()
    self.compile_model()
    self.build_checkpoint()

  def __str__(self):
    return '\n'.join([
      f'SegFormer checkpoint: {self.checkpoint}',
      f'labels: {self.id2label}',
      f'fire_masks: {self.fire_masks}'
    ])

  def build_paths(self):
    self.base_directory = wildfire_drive_path
    self.model_directory = 'segformer_model'
    self.model_path = os.path.join(self.base_directory, self.model_directory)
    self.checkpoint_directory = 'segformer_checkpoint'
    self.checkpoint_path = os.path.join(
        self.base_directory, self.checkpoint_directory, 'segformer')

  def build_model(self):
    self.build_id2label()
    self.build_label2id()
    self.num_labels = len(self.id2label)
    self.model = TFSegformerForSemanticSegmentation.from_pretrained(
        self.checkpoint,
        num_labels=self.num_labels,
        id2label=self.id2label,
        label2id=self.label2id,
        ignore_mismatched_sizes=True,
    )
    self.model.summary()

  def build_id2label(self):
    max_fire_mask = max(self.fire_masks)
    id2label = {}
    for i in range(max_fire_mask + 1):
      id2label[i] = str(i)
    self.id2label = id2label

  def build_label2id(self):
    max_fire_mask = max(self.fire_masks)
    label2id = {}
    for i in range(max_fire_mask + 1):
      label2id[str(i)] = i
    self.label2id = label2id

  def compile_model(self):
    self.model.compile(optimizer='adam')

  def build_checkpoint(self):
    self.checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        self.checkpoint_path, save_weights_only=True)
    checkpoint_directory = os.path.dirname(self.checkpoint_path)
    latest_checkpoint_path = tf.train.latest_checkpoint(checkpoint_directory)
    if latest_checkpoint_path:
      print(f'Load checkpoint: {latest_checkpoint_path}')
      self.model.load_weights(latest_checkpoint_path)

  def train_and_save_model(self, epochs=1):
    self.build_checkpoint()
    self.train_model(epochs)
    self.save_model()

  def train_model(self, epochs=1):
    train_dataset = self.dataset_builder.train_dataset
    test_dataset = self.dataset_builder.test_dataset
    self.history = self.model.fit(
        train_dataset,
        validation_data=test_dataset,
        epochs=epochs,
        callbacks=[self.checkpoint_callback]
    )

  def save_model(self):
    self.model.save(self.model_path)

  def load_model(self):
    self.model = tf.keras.models.load_model(self.model_path)

  def run_predict(self, inputs, labels=None):
    image_size = self.image_size
    probabilities = self.model.predict(inputs).logits
    predictions = tf.math.argmax(probabilities, axis=1)
    predictions = predictions[0]
    predictions = tf.expand_dims(predictions, -1)
    predictions = tf.image.resize(predictions, (image_size, image_size),
                                  method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    predictions = tf.squeeze(predictions)
    predictions = predictions.numpy().astype(np.uint8)
    if labels is not None:
      labels = labels[0]
      if tf.is_tensor(labels):
        labels = labels.numpy().astype(np.uint8)
    return predictions, labels


segformer_model_builder = SegFormerModelBuilder()
print(segformer_model_builder)

In [ ]:
#@title Train the model
#segformer_model_builder.train_and_save_model(epochs=1)
#print(os.listdir(segformer_model_builder.model_path))

In [ ]:
#@title Analyze Model
class SegFormerModelAnalyzer(UnetModelAnalyzer):

  def __init__(self):
    super(SegFormerModelAnalyzer, self).__init__(
        dataset_builder=segformer_dataset_builder,
        model_builder=segformer_model_builder)

  def build_dataset(self, max_examples=5):
    all_examples = []
    dataset = self.dataset_builder.test_dataset.unbatch().batch(1)
    for example in dataset:
      if max_examples <= 0:
        break
      max_examples -= 1
      inputs, labels = example["pixel_values"], example["labels"]
      all_examples.append([inputs, labels])
    return all_examples

segformer_model_analyzer = SegFormerModelAnalyzer()
segformer_model_analyzer.run_analysis()